In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install kaggle -q
!pip install catboost -q

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c brist1d

In [ ]:
!mkdir -p ./data/brist-1d
!unzip brist1d.zip -d ./data/brist-1d
!ls ./data/brist-1d

In [ ]:
import numpy as np
import pandas as pd


import lightgbm as lgb
import catboost as cb
import xgboost as xgb

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GroupKFold

from math import sqrt

import shutil


import os
import joblib

import scipy.stats as stats



pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option("display.max_rows", None)

In [ ]:
train = pd.read_csv('/content/data/brist-1d/train.csv')
train.head()


In [ ]:
class CONFIG:

    TARGET_COL = 'bg+1+00'
    GROUP_COL = 'p_num'
    N_SPLITS = 5
    # SEED_LIST = [2,12,22,32,42,52,62,72,82,92,102,112,122,132,142,152]
    SEED_LIST = [42]

    ERR = 1e-5

    BG_LOW = 3.9
    BG_HIGH = 10.0


    TRAIN_CATBOOST = False
    TRAIN_XGB = True
    TRAIN_LGB = False

    early_stop = 200

    catboost_params = {
        # 'random_state': 42,  # This will be updated per seed
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'learning_rate': 0.025,
        'iterations': 1000,
        'task_type': 'CPU',
        'depth': 7,
    }

    xgboost_params = {
        'random_state': 42,  # This will be updated per seed
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'learning_rate': 0.025,
        'tree_method': 'hist',
        'device': 'cpu',
        'max_depth': 7,
    }


    lgb_params = {
        # 'random_state': 42,  # This will be updated per seed
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'num_threads': 8,
        'learning_rate': 0.025,
        # 'n_estimators': 20000,
        'device': 'cpu',
        'max_depth': 5,
        'num_leaves': 64,
        # 'min_data_in_leaf': 50,
        # 'lambda_l1': 30,
        # 'lambda_l2': 10,
        'verbosity': -1
    }

In [ ]:
def _extract_time(col: str) -> float:
    time_str = col.split('-')[-1] if '-' in col else col
    hours, minutes = map(int, time_str.split('+'))
    return hours + minutes/60

def _within_window(col: str, window: str) -> bool:
    try:
        col_time = _extract_time(col)
        window_time = _extract_time(window)
        return col_time <= window_time
    except:
        return False

In [ ]:
# @title feature engineering

def calculate_patient_features(df: pd.DataFrame) -> pd.DataFrame:

    bg_columns = [col for col in df.columns if col.startswith('bg-')]
    insulin_columns = [col for col in df.columns if col.startswith('insulin-')]
    hr_columns = [col for col in df.columns if col.startswith('hr-')]
    steps_columns = [col for col in df.columns if col.startswith('steps-')]
    cals_columns = [col for col in df.columns if col.startswith('cals-')]


    df['bg_sum_5hr'] = df[bg_columns].sum(axis=1)
    df['bg_mean_5hr'] = df['bg_sum_5hr'] / len(bg_columns)

    df['insulin_sum_5hr'] = df[insulin_columns].sum(axis=1)
    df['insulin_mean_5hr'] = df['insulin_sum_5hr'] / len(insulin_columns)

    df['hr_sum_5hr'] = df[hr_columns].sum(axis=1)
    df['hr_mean_5hr'] = df['hr_sum_5hr'] / len(hr_columns)

    df['steps_sum_5hr'] = df[steps_columns].sum(axis=1)
    df['steps_mean_5hr'] = df['steps_sum_5hr'] / len(steps_columns)

    df['cals_sum_5hr'] = df[cals_columns].sum(axis=1)
    df['cals_mean_5hr'] = df['cals_sum_5hr'] / len(cals_columns)

    return df

In [ ]:
def preprocess(df):

  df.columns = df.columns.str.replace(':', '+', regex=False)

  df['time'] = pd.to_datetime(df['time'])
  df['hour'] = df['time'].dt.hour
  df['minute'] = df['time'].dt.minute

  df['part_of_day'] = pd.cut(df['hour'],
                                 bins=[-np.inf, 6, 12, 18, np.inf],
                                 labels=['night', 'morning', 'afternoon', 'evening'])

  df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
  df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)

  cat_cols = df.select_dtypes(include='object').columns.tolist()
  numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

  df[cat_cols] = df[cat_cols].fillna('Unknown').astype('category')
  df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')

  df = calculate_patient_features(df)

  df.drop(df.filter(regex='carbs|activity').columns, axis=1, inplace=True)

  df.drop(['time'], axis=1, inplace=True)
  df.drop(['id'], axis=1, inplace=True)

  return df

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train['p_num'].value_counts()

In [ ]:
# @title train_catboost
def train_catboost(data, model_save_dir='catboost_models'):

    if os.path.exists(model_save_dir):
        shutil.rmtree(model_save_dir)

    # Create the directory
    os.makedirs(model_save_dir)

    X = data.drop([CONFIG.TARGET_COL, CONFIG.GROUP_COL], axis=1)
    y = data[CONFIG.TARGET_COL]
    groups = data[CONFIG.GROUP_COL]

    cv = GroupKFold(n_splits=CONFIG.N_SPLITS)
    all_oof_preds = np.zeros(X.shape[0])
    all_metrics = []
    all_models = []
    feature_importances = np.zeros(X.shape[1])

    seed_oof_preds = []

    cat_features = [X.columns.get_loc(col) for col in X.select_dtypes(include=['category'])]

    for seed in CONFIG.SEED_LIST:
        print(f'Training with seed: {seed} for feature importance calculation...')
        CONFIG.catboost_params['random_state'] = seed

        oof_preds = np.zeros(X.shape[0])
        metrics = []
        models = []

        for fi, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups)):

            print(f'Fold {fi + 1}/{CONFIG.N_SPLITS} with seed {seed}...')

            model = cb.CatBoostRegressor(**CONFIG.catboost_params)

            model.fit(X.iloc[train_idx], y.iloc[train_idx],
                      eval_set=(X.iloc[valid_idx], y.iloc[valid_idx]),
                      use_best_model=True,
                      # early_stopping_rounds=Config.early_stop,
                      cat_features=cat_features,
                      verbose=200)

            preds = model.predict(X.iloc[valid_idx])
            oof_preds[valid_idx] = preds

            rmse = sqrt(mean_squared_error(y.iloc[valid_idx], preds))
            metrics.append(rmse)
            print(f'Fold {fi + 1} RMSE: {rmse:.4f}')

            models.append(model)

            model_save_path = os.path.join(model_save_dir, f'catboost_model_seed_{seed}_fold_{fi}.pkl')
            joblib.dump(model, model_save_path)

            feature_importances += model.get_feature_importance()

        seed_oof_preds.append(oof_preds)
        all_models.append(models)
        all_metrics.append(metrics)

    oof_preds_avg = np.mean(seed_oof_preds, axis=0)
    oof_rmse = sqrt(mean_squared_error(y, oof_preds_avg))

    all_fold_rmses = np.array([rmse for metrics in all_metrics for rmse in metrics])

    print(f'Average RMSE across seeds: {np.mean([np.mean(m) for m in all_metrics]):.4f}')
    print(f'STD of RMSE across folds: {np.std(all_fold_rmses):.4f}')
    print(f'OOF RMSE across seeds: {oof_rmse:.4f}')

        # t-statistic and p-value calculation
    seed_fold_rmses = np.array([[rmse for rmse in metrics] for metrics in all_metrics])

    for i in range(len(CONFIG.SEED_LIST)):
        for j in range(i + 1, len(CONFIG.SEED_LIST)):
            t_stat, p_value = stats.ttest_rel(seed_fold_rmses[i], seed_fold_rmses[j])
            print(f'T-statistic between seed {CONFIG.SEED_LIST[i]} and {CONFIG.SEED_LIST[j]}: {t_stat:.4f}, p-value: {p_value:.4f}')

    return all_models


In [ ]:
# @title train_xgboost
def train_xgboost(data):

    X = data.drop([CONFIG.TARGET_COL, CONFIG.GROUP_COL], axis=1)
    y = data[CONFIG.TARGET_COL]
    groups = data[CONFIG.GROUP_COL]

    cv = GroupKFold(n_splits=CONFIG.N_SPLITS)
    all_oof_preds = np.zeros(X.shape[0])
    all_metrics = []
    all_models = []
    feature_importances = np.zeros(X.shape[1])

    seed_oof_preds = []

    cat_cols = X.select_dtypes(include='category').columns

    for seed in CONFIG.SEED_LIST:
        print(f'Training with seed: {seed} for feature importance calculation...')
        # CONFIG.xgboost_params['random_state'] = seed

        oof_preds = np.zeros(X.shape[0])
        metrics = []
        models = []

        for fi, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups)):

            # if fi != 0:
            #     continue

            print(f'Fold {fi + 1}/{CONFIG.N_SPLITS} with seed {seed}...')

            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx], enable_categorical=True)
            dvalid = xgb.DMatrix(X.iloc[valid_idx], label=y.iloc[valid_idx], enable_categorical=True)

            model = xgb.train(
                params=CONFIG.xgboost_params,
                dtrain=dtrain,
                num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                verbose_eval=200,
                early_stopping_rounds=CONFIG.early_stop,
            )

            preds = model.predict(xgb.DMatrix(X.iloc[valid_idx], enable_categorical=True))
            oof_preds[valid_idx] = preds

            rmse = sqrt(mean_squared_error(y.iloc[valid_idx], preds))
            metrics.append(rmse)
            print(f'Fold {fi + 1} RMSE: {rmse:.4f}')

            models.append(model)

        seed_oof_preds.append(oof_preds)
        all_models.append(models)
        all_metrics.append(metrics)

    oof_preds_avg = np.mean(seed_oof_preds, axis=0)
    oof_rmse = sqrt(mean_squared_error(y, oof_preds_avg))

    all_fold_rmses = np.array([rmse for metrics in all_metrics for rmse in metrics])

    print(f'Average RMSE across seeds: {np.mean([np.mean(m) for m in all_metrics]):.4f}')
    print(f'STD of RMSE across folds: {np.std(all_fold_rmses):.4f}')
    print(f'OOF RMSE across seeds: {oof_rmse:.4f}')

    return all_models

In [ ]:
# @title train_lgb
def train_lgb(data):
    X = data.drop([CONFIG.TARGET_COL, CONFIG.GROUP_COL], axis=1)
    y = data[CONFIG.TARGET_COL]
    groups = data[CONFIG.GROUP_COL]

    cv = GroupKFold(n_splits=CONFIG.N_SPLITS)
    all_oof_preds = np.zeros(X.shape[0])
    all_metrics = []
    all_models = []

    seed_oof_preds = []

    for seed in CONFIG.SEED_LIST:
        print(f'Training with seed: {seed} for feature importance calculation...')

        oof_preds = np.zeros(X.shape[0])
        metrics = []
        models = []

        for fi, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups)):

            print(f'Fold {fi + 1}/{CONFIG.N_SPLITS} with seed {seed}...')
            dtrain = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx])
            dvalid = lgb.Dataset(X.iloc[valid_idx], label=y.iloc[valid_idx], reference=dtrain)

            model = lgb.train(
                params={**CONFIG.lgb_params, 'random_state': seed},
                train_set=dtrain,
                valid_sets=[dtrain, dvalid],
                num_boost_round=1000,
                callbacks=[lgb.early_stopping(stopping_rounds=CONFIG.early_stop), lgb.log_evaluation(period=200)]
            )


            preds = model.predict(X.iloc[valid_idx])
            oof_preds[valid_idx] = preds

            rmse = sqrt(mean_squared_error(y.iloc[valid_idx], preds))
            metrics.append(rmse)
            print(f'Fold {fi + 1} RMSE: {rmse:.4f}')

            models.append(model)

        seed_oof_preds.append(oof_preds)
        all_models.append(models)
        all_metrics.append(metrics)

    oof_preds_avg = np.mean(seed_oof_preds, axis=0)
    oof_rmse = sqrt(mean_squared_error(y, oof_preds_avg))

    all_fold_rmses = np.array([rmse for metrics in all_metrics for rmse in metrics])

    print(f'Average RMSE across seeds: {np.mean([np.mean(m) for m in all_metrics]):.4f}')
    print(f'STD of RMSE across folds: {np.std(all_fold_rmses):.4f}')
    print(f'OOF RMSE across seeds: {oof_rmse:.4f}')


    seed_fold_rmses = np.array([[rmse for rmse in metrics] for metrics in all_metrics])

    for i in range(len(CONFIG.SEED_LIST)):
        for j in range(i + 1, len(CONFIG.SEED_LIST)):
            t_stat, p_value = stats.ttest_rel(seed_fold_rmses[i], seed_fold_rmses[j])
            print(f'T-statistic between seed {CONFIG.SEED_LIST[i]} and {CONFIG.SEED_LIST[j]}: {t_stat:.4f}, p-value: {p_value:.4f}')

    return all_models


In [ ]:
%%time
train = preprocess(train)

train.shape

In [ ]:
train.head()

In [ ]:
%%time

if CONFIG.TRAIN_CATBOOST:
    catboost_models = train_catboost(train)

if CONFIG.TRAIN_XGB:
    xgboost_models = train_xgboost(train)


if CONFIG.TRAIN_LGB:
    lgb_models = train_lgb(train)

In [ ]:


#lgbm 2.06
# std: 0.144

# xgboost
# STD of RMSE across folds: 0.1352
# OOF RMSE across seeds: 2.0997

In [ ]:
test = pd.read_csv('/content/data/brist-1d/test.csv')
test.head()

In [ ]:
def infer_xgboost(test_data, all_models):

    X_test = test_data.drop(CONFIG.GROUP_COL, axis=1)
    cat_cols = X_test.select_dtypes(include='category').columns

    all_test_preds = np.zeros(X_test.shape[0])

    for seed_models in all_models:
        seed_preds = np.zeros(X_test.shape[0])

        for model in seed_models:
            dtest = xgb.DMatrix(X_test, enable_categorical=True)
            fold_preds = model.predict(dtest)
            seed_preds += fold_preds / len(seed_models)  

        all_test_preds += seed_preds / len(all_models)  

    return all_test_preds

In [ ]:
def infer_catboost(test_data, all_models):

    X_test = test_data.drop(CONFIG.GROUP_COL, axis=1)
    cat_cols = X_test.select_dtypes(include='category').columns

    all_test_preds = np.zeros(X_test.shape[0])

    for seed_models in all_models:
        seed_preds = np.zeros(X_test.shape[0])
        for model in seed_models:
            fold_preds = model.predict(X_test)
            seed_preds += fold_preds / len(seed_models)  
        all_test_preds += seed_preds / len(all_models)  

    return all_test_preds

In [ ]:
def infer_lightgbm(test_data, all_models):

    X_test = test_data.drop(CONFIG.GROUP_COL, axis=1)
    cat_cols = X_test.select_dtypes(include='category').columns

    all_test_preds = np.zeros(X_test.shape[0])

    for seed_models in all_models:
        seed_preds = np.zeros(X_test.shape[0])
        for model in seed_models:
            fold_preds = model.predict(X_test)
            seed_preds += fold_preds / len(seed_models)  
        all_test_preds += seed_preds / len(all_models)  
    return all_test_preds

In [ ]:
test = preprocess(test)

test.shape

In [ ]:
xgb_predictions = infer_xgboost(test, xgboost_models)
cb_predictions = infer_catboost(test, catboost_models)
lgb_predictions = infer_lightgbm(test, lgb_models)

In [ ]:
sub = pd.read_csv('/content/data/brist-1d/sample_submission.csv')
sub['bg+1:00'] = xgb_predictions
sub.to_csv('xgb_submission.csv', index=False)
print(sub.head())

In [ ]:
sub = pd.read_csv('/content/data/brist-1d/sample_submission.csv')
sub['bg+1:00'] = cb_predictions
sub.to_csv('cb_submission.csv', index=False)
print(sub.head())

In [ ]:
sub = pd.read_csv('/content/data/brist-1d/sample_submission.csv')
sub['bg+1:00'] = lgb_predictions
sub.to_csv('lgb_submission.csv', index=False)
print(sub.head())